# Анализ сервиса вопросов и ответов по программированию

<b>Цель исследования</b> - анализ сервиса вопросов и ответов по программированию.

<b>Исходные данные для исследования</b>  - самостоятельная работа основана на базе данных StackOverflow — сервиса вопросов и ответов о программировании с данными о постах за 2008 год. 

<div align='right'>Проектную работу выполнил студент Гагиев Денис. </div>

---

**Задача 1** - найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [14]:
'''
SELECT COUNT(title)
FROM stackoverflow.posts
WHERE score > 300 OR favorites_count >= 100
'''

'\nSELECT COUNT(title)\nFROM stackoverflow.posts\nWHERE score > 300 OR favorites_count >= 100\n'

**Задача 2** - сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [15]:
'''
WITH df AS (
SELECT
    DATE_TRUNC('day', creation_date)::DATE AS posts_date,
    COUNT(*) AS posts_cnt
FROM stackoverflow.posts
WHERE creation_date::DATE BETWEEN '01.11.2008' AND '18.11.2008' AND post_type_id = 1
GROUP BY 1)

SELECT
    ROUND(AVG(posts_cnt))
FROM df
'''

"\nWITH df AS (\nSELECT\n    DATE_TRUNC('day', creation_date)::DATE AS posts_date,\n    COUNT(*) AS posts_cnt\nFROM stackoverflow.posts\nWHERE creation_date::DATE BETWEEN '01.11.2008' AND '18.11.2008' AND post_type_id = 1\nGROUP BY 1)\n\nSELECT\n    ROUND(AVG(posts_cnt))\nFROM df\n"

**Задача 3** - Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [16]:
'''
SELECT
    COUNT(DISTINCT u.id)
FROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id=b.user_id
WHERE u.creation_date::DATE = b.creation_date::DATE
'''

'\nSELECT\n    COUNT(DISTINCT u.id)\nFROM stackoverflow.users u JOIN stackoverflow.badges b ON u.id=b.user_id\nWHERE u.creation_date::DATE = b.creation_date::DATE\n'

**Задача 4** - Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [17]:
'''
SELECT
    COUNT(DISTINCT p.title)
FROM stackoverflow.votes v
LEFT JOIN stackoverflow.posts p ON v.post_id = p.id
WHERE p.user_id = '3043'
'''

"\nSELECT\n    COUNT(DISTINCT p.title)\nFROM stackoverflow.votes v\nLEFT JOIN stackoverflow.posts p ON v.post_id = p.id\nWHERE p.user_id = '3043'\n"

**Задача 5** - Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [18]:
'''
SELECT *,
    RANK() OVER (ORDER BY id desc )
FROM stackoverflow.vote_types
ORDER BY id 
'''

'\nSELECT *,\n    RANK() OVER (ORDER BY id desc )\nFROM stackoverflow.vote_types\nORDER BY id \n'

**Задача 6** - Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [19]:
'''
SELECT
    v.user_id,
    COUNT(*)
FROM stackoverflow.votes v
JOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id
WHERE vt.name = 'Close'
GROUP BY 1
ORDER BY 2 DESC, 1 DESC
LIMIT 10
'''

"\nSELECT\n    v.user_id,\n    COUNT(*)\nFROM stackoverflow.votes v\nJOIN stackoverflow.vote_types vt ON v.vote_type_id = vt.id\nWHERE vt.name = 'Close'\nGROUP BY 1\nORDER BY 2 DESC, 1 DESC\nLIMIT 10\n"

**Задача 7** - Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
- идентификатор пользователя;
- число значков;
- место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.
Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [20]:
'''
WITH df AS
    (SELECT
        u.id,
        COUNT(*) AS badges_cnt
    FROM stackoverflow.users u
    RIGHT JOIN stackoverflow.badges b ON u.id=b.user_id
    WHERE b.creation_date::DATE BETWEEN ('15.11.2008') AND ('15.12.2008')
    GROUP BY 1
    ORDER BY 2 DESC, 1
    LIMIT 10)
    
SELECT *,
    DENSE_RANK() OVER (ORDER BY badges_cnt DESC)
FROM df
'''

"\nWITH df AS\n    (SELECT\n        u.id,\n        COUNT(*) AS badges_cnt\n    FROM stackoverflow.users u\n    RIGHT JOIN stackoverflow.badges b ON u.id=b.user_id\n    WHERE b.creation_date::DATE BETWEEN ('15.11.2008') AND ('15.12.2008')\n    GROUP BY 1\n    ORDER BY 2 DESC, 1\n    LIMIT 10)\n    \nSELECT *,\n    DENSE_RANK() OVER (ORDER BY badges_cnt DESC)\nFROM df\n"

**Задача 8** - Сколько в среднем очков получает пост каждого пользователя?

Сформируйте таблицу из следующих полей:
- заголовок поста;
- идентификатор пользователя;
- число очков поста;
- среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [21]:
'''
SELECT
    title,
    user_id,
    score,
    ROUND(AVG(score) OVER (PARTITION BY user_id))
FROM stackoverflow.posts
WHERE title != '' AND score != 0
'''

"\nSELECT\n    title,\n    user_id,\n    score,\n    ROUND(AVG(score) OVER (PARTITION BY user_id))\nFROM stackoverflow.posts\nWHERE title != '' AND score != 0\n"

**Задача 9** - Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [22]:
'''
SELECT
    title
FROM stackoverflow.posts
WHERE user_id = '22656' AND title != ''
'''

"\nSELECT\n    title\nFROM stackoverflow.posts\nWHERE user_id = '22656' AND title != ''\n"

**Задача 10** - Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). 
Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
- пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
- пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
- пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [23]:
'''
SELECT
    id,
    views,
    CASE
        WHEN views >= 350 THEN 1
        WHEN views < 100 THEN 3
        ELSE 2
    END AS group
FROM stackoverflow.users
WHERE location LIKE '%United States' AND views != 0
'''

"\nSELECT\n    id,\n    views,\n    CASE\n        WHEN views >= 350 THEN 1\n        WHEN views < 100 THEN 3\n        ELSE 2\n    END AS group\nFROM stackoverflow.users\nWHERE location LIKE '%United States' AND views != 0\n"

**Задача 11** - Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [24]:
'''
WITH df AS (
    SELECT
        df1.id,
        df1.groupi,
        df1.views,
        MAX(df1.views) OVER (PARTITION BY df1.groupi)
    FROM (
        SELECT
            id,
            views,
            CASE
                WHEN views >= 350 THEN 1
                WHEN views < 100 THEN 3
                ELSE 2
            END AS groupi
        FROM stackoverflow.users
        WHERE location LIKE '%United States%' AND views != 0) AS df1)

SELECT
    id, groupi, views
FROM df
WHERE views = max
ORDER BY 3 DESC, 1
'''

"\nWITH df AS (\n    SELECT\n        df1.id,\n        df1.groupi,\n        df1.views,\n        MAX(df1.views) OVER (PARTITION BY df1.groupi)\n    FROM (\n        SELECT\n            id,\n            views,\n            CASE\n                WHEN views >= 350 THEN 1\n                WHEN views < 100 THEN 3\n                ELSE 2\n            END AS groupi\n        FROM stackoverflow.users\n        WHERE location LIKE '%United States%' AND views != 0) AS df1)\n\nSELECT\n    id, groupi, views\nFROM df\nWHERE views = max\nORDER BY 3 DESC, 1\n"

**Задача 12** - Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. 
Сформируйте таблицу с полями:
- номер дня;
- число пользователей, зарегистрированных в этот день;
- сумму пользователей с накоплением.

In [25]:
'''
WITH df AS (SELECT
    EXTRACT(DAY FROM creation_date::DATE) AS days,
    COUNT(id) AS users_up
FROM stackoverflow.users
    WHERE creation_date::DATE BETWEEN '01.11.2008' AND '30.11.2008'
GROUP BY 1)

SELECT *,
    SUM(users_up) OVER (order by days)
FROM df
'''

"\nWITH df AS (SELECT\n    EXTRACT(DAY FROM creation_date::DATE) AS days,\n    COUNT(id) AS users_up\nFROM stackoverflow.users\n    WHERE creation_date::DATE BETWEEN '01.11.2008' AND '30.11.2008'\nGROUP BY 1)\n\nSELECT *,\n    SUM(users_up) OVER (order by days)\nFROM df\n"

**Задача 13** - Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. 
Отобразите:
- идентификатор пользователя;
- разницу во времени между регистрацией и первым постом.

In [26]:
'''
WITH df AS (
    SELECT
        DISTINCT user_id,
        MIN(creation_date) OVER (PARTITION BY user_id) min_post
    FROM stackoverflow.posts)

SELECT
    u.id,
    (post.min_post - u.creation_date )
FROM df AS post
JOIN stackoverflow.users AS u ON u.id = post.user_id
'''

'\nWITH df AS (\n    SELECT\n        DISTINCT user_id,\n        MIN(creation_date) OVER (PARTITION BY user_id) min_post\n    FROM stackoverflow.posts)\n\nSELECT\n    u.id,\n    (post.min_post - u.creation_date )\nFROM df AS post\nJOIN stackoverflow.users AS u ON u.id = post.user_id\n'

**Задача 14** - Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [27]:
'''
SELECT
    DATE_TRUNC('month', creation_date)::DATE AS monthh,
    SUM(views_count)
FROM stackoverflow.posts
WHERE EXTRACT(YEAR FROM creation_date::DATE) = '2008'
GROUP BY 1
ORDER BY 2 DESC
'''

"\nSELECT\n    DATE_TRUNC('month', creation_date)::DATE AS monthh,\n    SUM(views_count)\nFROM stackoverflow.posts\nWHERE EXTRACT(YEAR FROM creation_date::DATE) = '2008'\nGROUP BY 1\nORDER BY 2 DESC\n"

**Задача 15** - Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [28]:
'''
SELECT
    u.display_name,
    COUNT(DISTINCT p.user_id)
FROM stackoverflow.posts AS p
JOIN stackoverflow.users AS u ON p.user_id = u.id
JOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id
WHERE p.creation_date::DATE BETWEEN u.creation_date::DATE AND (u.creation_date::DATE + INTERVAL '1 month')
    AND pt.type LIKE '%Answer%'
GROUP BY 1
HAVING COUNT(p.user_id) > 100
ORDER BY 1
'''

"\nSELECT\n    u.display_name,\n    COUNT(DISTINCT p.user_id)\nFROM stackoverflow.posts AS p\nJOIN stackoverflow.users AS u ON p.user_id = u.id\nJOIN stackoverflow.post_types AS pt ON pt.id = p.post_type_id\nWHERE p.creation_date::DATE BETWEEN u.creation_date::DATE AND (u.creation_date::DATE + INTERVAL '1 month')\n    AND pt.type LIKE '%Answer%'\nGROUP BY 1\nHAVING COUNT(p.user_id) > 100\nORDER BY 1\n"

**Задача 16** - Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [29]:
'''
WITH df AS (
    SELECT u.id
    FROM stackoverflow.users AS u
    JOIN stackoverflow.posts AS p ON u.id = p.user_id
    WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01'
              AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'
    GROUP BY 1
    HAVING COUNT(u.id) > 0    
)

SELECT
    DATE_TRUNC('month', p.creation_date)::date,
    COUNT(DISTINCT p.id)
FROM stackoverflow.posts p
WHERE p.user_id IN (SELECT * FROM df)
    AND EXTRACT(YEAR FROM p.creation_date::DATE) = 2008
GROUP BY 1 
ORDER BY 1 DESC 
'''

"\nWITH df AS (\n    SELECT u.id\n    FROM stackoverflow.users AS u\n    JOIN stackoverflow.posts AS p ON u.id = p.user_id\n    WHERE DATE_TRUNC('month', u.creation_date)::date = '2008-09-01'\n              AND DATE_TRUNC('month', p.creation_date)::date = '2008-12-01'\n    GROUP BY 1\n    HAVING COUNT(u.id) > 0    \n)\n\nSELECT\n    DATE_TRUNC('month', p.creation_date)::date,\n    COUNT(DISTINCT p.id)\nFROM stackoverflow.posts p\nWHERE p.user_id IN (SELECT * FROM df)\n    AND EXTRACT(YEAR FROM p.creation_date::DATE) = 2008\nGROUP BY 1 \nORDER BY 1 DESC \n"

**Задача 17** - Используя данные о постах, выведите несколько полей:
- идентификатор пользователя, который написал пост;
- дата создания поста;
- количество просмотров у текущего поста;
- сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [30]:
'''
SELECT
    user_id,
    creation_date,
    views_count,
    SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)
FROM stackoverflow.posts
'''

'\nSELECT\n    user_id,\n    creation_date,\n    views_count,\n    SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date)\nFROM stackoverflow.posts\n'

**Задача 18** - Сколько в среднем дней в период с 1 по 7 декабря 2008 года включительно пользователи взаимодействовали с платформой? Для каждого пользователя отберите дни, в которые он или она опубликовали хотя бы один пост. Нужно получить одно целое число — не забудьте округлить результат.

In [31]:
'''
WITH df AS
(SELECT
    user_id,
    COUNT(DISTINCT creation_date::DATE) AS cnt
FROM stackoverflow.posts
WHERE creation_date::DATE BETWEEN '01.12.2008' AND '07.12.2008'
GROUP BY 1)

SELECT
    ROUND(AVG(cnt))
FROM df
'''

"\nWITH df AS\n(SELECT\n    user_id,\n    COUNT(DISTINCT creation_date::DATE) AS cnt\nFROM stackoverflow.posts\nWHERE creation_date::DATE BETWEEN '01.12.2008' AND '07.12.2008'\nGROUP BY 1)\n\nSELECT\n    ROUND(AVG(cnt))\nFROM df\n"

**Задача 19** - На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
- номер месяца;
- количество постов за месяц;
- процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.
Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [32]:
'''
WITH df AS (
SELECT
    EXTRACT('month' FROM creation_date::DATE),
    COUNT(DISTINCT id) AS cnt
FROM stackoverflow.posts
WHERE creation_date::DATE BETWEEN '01.09.2008' AND '31.12.2008'
GROUP BY 1)

SELECT
    *,
    ROUND((cnt::numeric / LAG(cnt) OVER () - 1) * 100, 2)
FROM df
'''

"\nWITH df AS (\nSELECT\n    EXTRACT('month' FROM creation_date::DATE),\n    COUNT(DISTINCT id) AS cnt\nFROM stackoverflow.posts\nWHERE creation_date::DATE BETWEEN '01.09.2008' AND '31.12.2008'\nGROUP BY 1)\n\nSELECT\n    *,\n    ROUND((cnt::numeric / LAG(cnt) OVER () - 1) * 100, 2)\nFROM df\n"

**Задача 20** - 
Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
- номер недели;
- дата и время последнего поста, опубликованного на этой неделе.

In [33]:
'''
WITH best AS (
    SELECT
        user_id,
        COUNT(DISTINCT id)
    FROM stackoverflow.posts
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1),
    df AS (
    SELECT
        user_id,
        creation_date AS date,
        EXTRACT('week' FROM creation_date::DATE) AS week
    FROM stackoverflow.posts
    WHERE user_id IN (SELECT user_id FROM best)
    AND DATE_TRUNC('month', creation_date)::DATE = '2008-10-01' )

SELECT DISTINCT week::numeric,
    MAX(date) OVER (PARTITION BY week)
FROM df
ORDER BY 1
'''

"\nWITH best AS (\n    SELECT\n        user_id,\n        COUNT(DISTINCT id)\n    FROM stackoverflow.posts\n    GROUP BY 1\n    ORDER BY 2 DESC\n    LIMIT 1),\n    df AS (\n    SELECT\n        user_id,\n        creation_date AS date,\n        EXTRACT('week' FROM creation_date::DATE) AS week\n    FROM stackoverflow.posts\n    WHERE user_id IN (SELECT user_id FROM best)\n    AND DATE_TRUNC('month', creation_date)::DATE = '2008-10-01' )\n\nSELECT DISTINCT week::numeric,\n    MAX(date) OVER (PARTITION BY week)\nFROM df\nORDER BY 1\n"